### 보스턴 주택 가격 데이터셋 다운받기

In [1]:
from tensorflow.keras.datasets.boston_housing import load_data

(x_train, y_train), (x_test, y_test) = load_data(path='boston_housing.npz',
                                                    test_split = 0.2,
                                                    seed = 777)

65536/57026 [==================================] - 0s 0us/step


### 데이터 형태 확인하기

In [2]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(404, 13) (404,)
(102, 13) (102,)


### 데이터 전처리 및 검증 데이터셋 만들기

In [3]:
import numpy as np

# 데이터 표준화
mean = np.mean(x_train, axis = 0)
std = np.std(x_train, axis = 0)

x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

In [4]:
# 검증 데이터셋 만들기
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.33, random_state = 777)

### 모델 구성하기

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
# 입력 데이터의 형태 반드시 명시
# 13차원의 데이터를 입력으로 받고, 64개의 출력을 가지는 첫 번째 Dense층
model.add(Dense(64, activation = 'relu', input_shape = (13, )))
model.add(Dense(32, activation = 'relu'))   # 32개의 출력을 가지는 Dense층
model.add(Dense(1))     # 하나의 값을 출력

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

### 학습하고 평가하기

In [7]:
history = model.fit(x_train, y_train, epochs = 300,
                    validation_data = (x_val, y_val))
model.evaluate(x_test, y_test)

Epoch 1/300
9/9 [==============================] - 1s 20ms/step - loss: 558.0796 - mae: 22.0168 - val_loss: 608.3313 - val_mae: 22.4736
Epoch 2/300
9/9 [==============================] - 0s 4ms/step - loss: 533.3172 - mae: 21.4113 - val_loss: 581.8182 - val_mae: 21.8556
Epoch 3/300
9/9 [==============================] - 0s 6ms/step - loss: 507.4832 - mae: 20.7797 - val_loss: 553.3861 - val_mae: 21.1870
Epoch 4/300
9/9 [==============================] - 0s 5ms/step - loss: 479.5453 - mae: 20.0829 - val_loss: 520.0281 - val_mae: 20.3994
Epoch 5/300
9/9 [==============================] - 0s 5ms/step - loss: 445.4734 - mae: 19.2406 - val_loss: 481.4133 - val_mae: 19.4645
Epoch 6/300
9/9 [==============================] - 0s 5ms/step - loss: 407.3647 - mae: 18.2387 - val_loss: 434.9320 - val_mae: 18.2936
Epoch 7/300
9/9 [==============================] - 0s 5ms/step - loss: 360.4289 - mae: 17.0023 - val_loss: 382.6127 - val_mae: 16.8892
Epoch 8/300
9/9 [==============================] - 0s 

[9.958894729614258, 2.2661900520324707]

### K-폴드 사용하기

In [14]:
from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
from sklearn.model_selection import KFold

In [15]:
(x_train, y_train), (x_test, y_test) = load_data(path='boston_housing.npz', test_split=0.2, seed=777)

In [16]:
# 데이터 표준화
mean = np.mean(x_train, axis = 0)
std = np.std(x_train, axis = 0)
# 여기까지는 전부 동일
x_train  = (x_train - mean) / std
x_test = (x_test - mean) / std

In [19]:
# K-폴드 진행
k = 3

# 주어진 데이터셋을 k만큼 등분
# 여기서는 3이므로 학습 데이터셋(404개)을 3등분하여
# 한 개는 검증셋으로, 나머지 두 개는 학습 데이터셋으로 활용
kfold = KFold(n_splits=k, shuffle=True)

In [20]:
# K-폴드 과정에서 재사용을 위해 모델을 반환하는 함수 정의
def get_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape = (13, )))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss = 'mse', metrics=['mae'])

    return model

In [22]:
mae_list = [] # 테스트셋을 평가한 후 결과 mae를 담을 리스트 선언

# k번 진행
for train_index, val_index in kfold.split(x_train):
    # 해당 인덱스는 무작위로 생성됨
    # 무작위로 생성해주는 것은 과대적합을 피할 수 있는 좋은 방법
    x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # 모델 불러오기
    model = get_model()

    model.fit(x_train_fold, y_train_fold, epochs= 300,
            validation_data = (x_val_fold, y_val_fold))

    _, test_mae = model.evaluate(x_test, y_test)
    mae_list.append(test_mae)


Epoch 1/300
9/9 [==============================] - 0s 17ms/step - loss: 547.6000 - mae: 21.6944 - val_loss: 606.5674 - val_mae: 22.6255
Epoch 2/300
9/9 [==============================] - 0s 4ms/step - loss: 505.1825 - mae: 20.7141 - val_loss: 561.1981 - val_mae: 21.5970
Epoch 3/300
9/9 [==============================] - 0s 6ms/step - loss: 465.0320 - mae: 19.7074 - val_loss: 510.9623 - val_mae: 20.4302
Epoch 4/300
9/9 [==============================] - 0s 6ms/step - loss: 418.6842 - mae: 18.5243 - val_loss: 450.0444 - val_mae: 18.9579
Epoch 5/300
9/9 [==============================] - 0s 5ms/step - loss: 363.4169 - mae: 17.0489 - val_loss: 378.9514 - val_mae: 17.1057
Epoch 6/300
9/9 [==============================] - 0s 6ms/step - loss: 300.5298 - mae: 15.1902 - val_loss: 299.0132 - val_mae: 14.8775
Epoch 7/300
9/9 [==============================] - 0s 5ms/step - loss: 233.4976 - mae: 12.9416 - val_loss: 217.5127 - val_mae: 12.2876
Epoch 8/300
9/9 [==============================] - 0s 

### K-폴드 결과 확인

In [23]:
print(mae_list)

[2.10693621635437, 2.4272124767303467, 2.0735957622528076]


In [24]:
print(np.mean(mae_list))

2.2025814851125083
